In [2]:
import numpy as np

In [3]:
import sys
!{sys.executable} -m pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/94/41/143e80c1f48f7e826380f382dff6caf93a698c77bf2e9e38290bbbfbfe42/xgboost-1.7.6-py3-none-macosx_10_15_x86_64.macosx_11_0_x86_64.macosx_12_0_x86_64.whl.metadata
Using cached xgboost-1.7.6-py3-none-macosx_10_15_x86_64.macosx_11_0_x86_64.macosx_12_0_x86_64.whl (1.8 MB)


### I
Each row of X is a 222d vector - each player accounts for half of it.
Distribution of 111 values:
- 109 values for the cards (if the card is used in the deck, it will be indicated with their levels). This means there are 8 non-zero values among these 109.
- Average elixir cost of deck 
- Elixir leaked
- Future improvement (in the works): Add additional features such as skill and experience (can be estimated by player win % and exp resp.)

In [3]:
X = np.load('X.npy')
X.shape

(642376, 222)

### Labels
Each row of X is a 222d vector - each player accounts for half of it.
Distribution of 111 values:
- 109 values for the cards (if the card is used in the deck, it will be indicated with their levels). This means there are 8 non-zero values among these 109.
- Average elixir cost of deck 
- Elixir leaked
- Future improvement (in the works): Add additional features such as skill and experience (can be estimated by player win % and exp resp.)

In [4]:
y = np.load('y.npy')
y.shape

(642376,)

In [5]:
# win %
y.mean()

0.519891776778709

## Preprocessing

In [6]:
# train test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [7]:
from sklearn.preprocessing import StandardScaler


# The standard score of a sample x is calculated as:
# z = (x - u) / s

# https://stats.stackexchange.com/questions/290958/logistic-regression-and-scaling-of-features

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Logistic regression

In [8]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(max_iter=1000, n_jobs=-1)
lr_model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, n_jobs=-1)

In [9]:
from sklearn.metrics import accuracy_score

train_accuracy = accuracy_score(lr_model.predict(X_train), y_train)
test_accuracy = accuracy_score(lr_model.predict(X_test), y_test)

print(f"train accuracy: {train_accuracy: .4f}")
print(f"test accuracy: {test_accuracy: .4f}")

train accuracy:  0.6382
test accuracy:  0.6394


## Random forest

note: scaling has no impact for tree models.

In [157]:
# parameter tuning

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(n_estimators = 100,
                            max_depth = 15, 
                            min_samples_split = 10, 
                            random_state = 0).fit(X_train, y_train)

# hyperparameter tuning...

# rf = RandomForestClassifier()

# print(rf.get_params())

# dic = {'bootstrap': [True, False], 
#        'max_depth': list(np.arange(10, 121, 10)) + [None], 
#        'max_features': ['sqrt', 'log2'],
#        'min_samples_leaf': [1, 2, 4],
#        'min_samples_split': [2, 5, 10],
#        'n_estimators': np.arange(10, 200, 10)
#       }

# # # random search first

# randomCV = RandomizedSearchCV(estimator=rf, param_distributions=dic, n_iter=10, n_jobs=-1, verbose=4)

# randomCV.fit(X_train, y_train)

# rf_model = randomCV.best_estimator_

# grid search...


In [158]:
from sklearn.metrics import accuracy_score

train_accuracy = accuracy_score(rf.predict(X_train), y_train)
test_accuracy = accuracy_score(rf.predict(X_test), y_test)

print(f"train accuracy: {train_accuracy: .4f}")
print(f"test accuracy: {test_accuracy: .4f}")

train accuracy:  0.7045
test accuracy:  0.6422


## XGBoost

In [9]:
from xgboost import XGBClassifier

In [10]:
X_train_fit, X_train_eval, y_train_fit, y_train_eval = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [12]:
xgb_model = XGBClassifier(n_estimators = 1000, learning_rate = 0.1,verbosity = 1, random_state = 0)
xgb_model.fit(X_train_fit,y_train_fit, eval_set = [(X_train_eval,y_train_eval)], early_stopping_rounds = 10)

/Users/joelfoo/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.68444
[1]	validation_0-logloss:0.67706
[2]	validation_0-logloss:0.67086
[3]	validation_0-logloss:0.66556
[4]	validation_0-logloss:0.66134
[5]	validation_0-logloss:0.65719
[6]	validation_0-logloss:0.65365
[7]	validation_0-logloss:0.65060
[8]	validation_0-logloss:0.64798
[9]	validation_0-logloss:0.64554
[10]	validation_0-logloss:0.64342
[11]	validation_0-logloss:0.64151
[12]	validation_0-logloss:0.63996
[13]	validation_0-logloss:0.63854
[14]	validation_0-logloss:0.63723
[15]	validation_0-logloss:0.63595
[16]	validation_0-logloss:0.63477
[17]	validation_0-logloss:0.63373
[18]	validation_0-logloss:0.63284
[19]	validation_0-logloss:0.63201
[20]	validation_0-logloss:0.63117
[21]	validation_0-logloss:0.63044
[22]	validation_0-logloss:0.62936
[23]	validation_0-logloss:0.62877
[24]	validation_0-logloss:0.62805
[25]	validation_0-logloss:0.62747
[26]	validation_0-logloss:0.62689
[27]	validation_0-logloss:0.62631
[28]	validation_0-logloss:0.62581
[29]	validation_0-loglos

[238]	validation_0-logloss:0.59914
[239]	validation_0-logloss:0.59913
[240]	validation_0-logloss:0.59909
[241]	validation_0-logloss:0.59906
[242]	validation_0-logloss:0.59900
[243]	validation_0-logloss:0.59898
[244]	validation_0-logloss:0.59888
[245]	validation_0-logloss:0.59881
[246]	validation_0-logloss:0.59879
[247]	validation_0-logloss:0.59875
[248]	validation_0-logloss:0.59873
[249]	validation_0-logloss:0.59867
[250]	validation_0-logloss:0.59865
[251]	validation_0-logloss:0.59863
[252]	validation_0-logloss:0.59859
[253]	validation_0-logloss:0.59853
[254]	validation_0-logloss:0.59851
[255]	validation_0-logloss:0.59848
[256]	validation_0-logloss:0.59842
[257]	validation_0-logloss:0.59839
[258]	validation_0-logloss:0.59837
[259]	validation_0-logloss:0.59836
[260]	validation_0-logloss:0.59834
[261]	validation_0-logloss:0.59831
[262]	validation_0-logloss:0.59828
[263]	validation_0-logloss:0.59825
[264]	validation_0-logloss:0.59822
[265]	validation_0-logloss:0.59821
[266]	validation_0-l

[473]	validation_0-logloss:0.59247
[474]	validation_0-logloss:0.59241
[475]	validation_0-logloss:0.59241
[476]	validation_0-logloss:0.59233
[477]	validation_0-logloss:0.59232
[478]	validation_0-logloss:0.59231
[479]	validation_0-logloss:0.59229
[480]	validation_0-logloss:0.59229
[481]	validation_0-logloss:0.59226
[482]	validation_0-logloss:0.59224
[483]	validation_0-logloss:0.59223
[484]	validation_0-logloss:0.59218
[485]	validation_0-logloss:0.59213
[486]	validation_0-logloss:0.59209
[487]	validation_0-logloss:0.59209
[488]	validation_0-logloss:0.59206
[489]	validation_0-logloss:0.59204
[490]	validation_0-logloss:0.59203
[491]	validation_0-logloss:0.59200
[492]	validation_0-logloss:0.59198
[493]	validation_0-logloss:0.59197
[494]	validation_0-logloss:0.59191
[495]	validation_0-logloss:0.59191
[496]	validation_0-logloss:0.59189
[497]	validation_0-logloss:0.59188
[498]	validation_0-logloss:0.59187
[499]	validation_0-logloss:0.59183
[500]	validation_0-logloss:0.59182
[501]	validation_0-l

[708]	validation_0-logloss:0.58847
[709]	validation_0-logloss:0.58847
[710]	validation_0-logloss:0.58847
[711]	validation_0-logloss:0.58845
[712]	validation_0-logloss:0.58844
[713]	validation_0-logloss:0.58843
[714]	validation_0-logloss:0.58838
[715]	validation_0-logloss:0.58837
[716]	validation_0-logloss:0.58836
[717]	validation_0-logloss:0.58835
[718]	validation_0-logloss:0.58833
[719]	validation_0-logloss:0.58833
[720]	validation_0-logloss:0.58831
[721]	validation_0-logloss:0.58829
[722]	validation_0-logloss:0.58829
[723]	validation_0-logloss:0.58829
[724]	validation_0-logloss:0.58829
[725]	validation_0-logloss:0.58828
[726]	validation_0-logloss:0.58828
[727]	validation_0-logloss:0.58825
[728]	validation_0-logloss:0.58823
[729]	validation_0-logloss:0.58823
[730]	validation_0-logloss:0.58822
[731]	validation_0-logloss:0.58821
[732]	validation_0-logloss:0.58820
[733]	validation_0-logloss:0.58820
[734]	validation_0-logloss:0.58820
[735]	validation_0-logloss:0.58819
[736]	validation_0-l

[943]	validation_0-logloss:0.58649
[944]	validation_0-logloss:0.58647
[945]	validation_0-logloss:0.58645
[946]	validation_0-logloss:0.58645
[947]	validation_0-logloss:0.58645
[948]	validation_0-logloss:0.58645
[949]	validation_0-logloss:0.58644
[950]	validation_0-logloss:0.58643
[951]	validation_0-logloss:0.58640
[952]	validation_0-logloss:0.58639
[953]	validation_0-logloss:0.58639
[954]	validation_0-logloss:0.58638
[955]	validation_0-logloss:0.58639
[956]	validation_0-logloss:0.58637
[957]	validation_0-logloss:0.58638
[958]	validation_0-logloss:0.58636
[959]	validation_0-logloss:0.58635
[960]	validation_0-logloss:0.58634
[961]	validation_0-logloss:0.58633
[962]	validation_0-logloss:0.58633
[963]	validation_0-logloss:0.58631
[964]	validation_0-logloss:0.58628
[965]	validation_0-logloss:0.58629
[966]	validation_0-logloss:0.58628
[967]	validation_0-logloss:0.58627
[968]	validation_0-logloss:0.58625
[969]	validation_0-logloss:0.58624
[970]	validation_0-logloss:0.58622
[971]	validation_0-l

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=0, ...)

In [13]:
xgb_model.best_iteration

994

In [14]:
from sklearn.metrics import accuracy_score

train_accuracy = accuracy_score(xgb_model.predict(X_train), y_train)
test_accuracy = accuracy_score(xgb_model.predict(X_test), y_test)

print(f"train accuracy: {train_accuracy: .4f}")
print(f"test accuracy: {test_accuracy: .4f}")
# some overfitting!

train accuracy:  0.7369
test accuracy:  0.6768


## Multilayer perceptron

In [137]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [126]:
# modified the model from https://arxiv.org/pdf/2106.01782.pdf
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(222, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        return self.linear_relu_stack(x)

In [127]:
class MatchDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).reshape(-1,1).float()
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [128]:
train_data = MatchDataset(X_train, y_train)
test_data = MatchDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [129]:
net = Net()
# https://medium.com/dejunhuang/learning-day-57-practical-5-loss-function-crossentropyloss-vs-bceloss-in-pytorch-softmax-vs-bd866c8a0d23
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)

In [130]:
sum([p.numel() for p in net.parameters()])

286721

In [131]:
i, l = next(iter(train_loader))
print(i.shape)
print(l.shape)

torch.Size([64, 222])
torch.Size([64, 1])


In [143]:
# with lr decay!
optimizer = optim.Adam(net.parameters(), lr=1e-6)

In [144]:
num_epochs = 2

for epoch in range(num_epochs):
    
    running_loss = 0.0
    
    for i, data in enumerate(train_loader):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        logits = net(inputs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if (i + 1) % 2000 == 0:
            print(f'epoch {epoch + 1}, iteration {i + 1}: running_loss {running_loss / 2000:.3f}')
            running_loss = 0.0

epoch 1, iteration 2000: running_loss 0.488
epoch 1, iteration 4000: running_loss 0.488
epoch 1, iteration 6000: running_loss 0.487
epoch 1, iteration 8000: running_loss 0.488
epoch 2, iteration 2000: running_loss 0.487
epoch 2, iteration 4000: running_loss 0.488
epoch 2, iteration 6000: running_loss 0.488
epoch 2, iteration 8000: running_loss 0.485


In [ ]:
# train accuracy, 0.544

In [145]:
correct = 0 
with torch.no_grad():
     for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        outputs = net(inputs)
        s = F.sigmoid(outputs)
        correct += (s.round() == labels).sum().item()

print(correct)
print(f'train accuracy: {correct / X_train.shape[0]}')

388604
train accuracy: 0.7561860284101966


In [ ]:
# 72% -> 74.6 -> 75.5 -> 75.61860284101966

In [146]:
correct = 0 
with torch.no_grad():
     for i, data in enumerate(test_loader, 0):
        inputs, labels = data
        outputs = net(inputs)
        s = F.sigmoid(outputs)
        correct += (s.round() == labels).sum().item()

print(correct)
print(f'train accuracy: {correct / X_test.shape[0]}')

86845
train accuracy: 0.6759628257417728


In [ ]:
# 0.6742115258881036 -> train accuracy: 0.6766789127930508 (best)  -> 0.6760406612908247, .6759628257417728 (starting to overfit)